In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

In [ ]:
tf.random.set_seed(42)

## Define Augmenters

In [ ]:
def get_gaussian_kernel(k: int, sigma: float) -> tf.Tensor:
    """
        Get kxk 2D gaussian kernel.
        args:
            k: int - Kernel size.
            sigma: float - Blur strength.
        returns:
            kernel_gauss: tf.Tensor - Gaussian kernel tensor.
    """

    x = tf.range(-k // 2 + 1, k // 2 + 1, dtype = tf.float32)
    
    x_gauss = tf.math.exp(-(tf.pow(x, 2) / (2 * tf.pow(sigma, 2))))
    x_gauss = x_gauss / tf.math.sqrt((2 * 3.14159 * tf.pow(sigma, 2)))

    kernel_gauss = tf.tensordot(x_gauss, x_gauss, axes = 0)
    x_scale = tf.reduce_sum(kernel_gauss)

    kernel_gauss = kernel_gauss / x_scale
    
    return kernel_gauss

In [ ]:
@tf.function
def apply_gaussian_noise(x_in: tf.Tensor) -> tf.Tensor:
    """
        Apply 2D gaussian blur to input tensor.

        - Blur input tensor 50% of the time
        - Randomly sample blur strength [0.1, 2.0]
        - Kernel size is 10% of the input tensor height / width

        args:
            x_in: tf.Tensor - Input tensor.
        returns:
            x_out: tf.Tensor - Augmented tensor.
    """

    roll_augment_flag = tf.random.uniform((),0,1)

    if roll_augment_flag >= 0.50:
        blur_strength = tf.random.uniform((),0.1,2.0)
        kernel_size = tf.cast(x_in.shape[1], tf.float32) * tf.constant(0.10)
        kernel_size = tf.cast(kernel_size, tf.int32)

        kernel = get_gaussian_kernel(kernel_size, blur_strength)
        kernel = kernel[..., tf.newaxis]
        kernel = tf.tile(
            kernel, tf.constant([1, 1, x_in.shape[-1]]))
        kernel = kernel[..., tf.newaxis]

        x_out = tf.nn.depthwise_conv2d(x_in, kernel, [1,1,1,1], 'SAME')
    else:
        x_out = x_in
    return x_out

## Test Gaussian Blur

In [ ]:
plt.imshow(get_gaussian_kernel(21,20.0).numpy())

In [ ]:
image_test = cv2.imread('lenna.png')
image_test = cv2.cvtColor(image_test, cv2.COLOR_BGR2RGB)
image_test = image_test / 255.

In [ ]:
plt.imshow(image_test)

In [ ]:
image_test_tensor = tf.convert_to_tensor(image_test, dtype = tf.float32)
# image_test_tensor = image_test_tensor[..., tf.newaxis]
image_test_tensor = image_test_tensor[tf.newaxis, ...]

In [ ]:
image_test_tensor.shape

In [ ]:
image_test_tensor_convolved = apply_gaussian_noise(image_test_tensor)
plt.imshow(
    np.squeeze(image_test_tensor_convolved.numpy()))

In [ ]:
image_test_tensor_convolved.shape

## Scrapyard

In [ ]:
x = tf.range(-9 // 2 + 1, 9 // 2 + 1, dtype = tf.float32)

In [ ]:
tf.tensordot(x,x, axes = 0)

In [ ]:
tensor_tmp = tf.math.exp(-tf.pow(x,2)/(2 * tf.pow(10.0,2)))
tensor_tmp = tensor_tmp / tf.math.sqrt(2 * 3.14159 * tf.pow(10.0,2))
tensor_tmp = tf.tensordot(tensor_tmp,tensor_tmp, axes = 0)
plt.imshow(tensor_tmp.numpy())

In [ ]:
# tf.tensordot(tensor_tmp,tensor_tmp, axes = 0)